In [1]:
import io
import logging
import subprocess
import sys
import warnings
from contextlib import suppress
from datetime import timedelta
from functools import reduce
from logging import StreamHandler
from os import chdir, getcwd, makedirs
from os.path import dirname, exists, join
from subprocess import CalledProcessError
from typing import List

import humanize
import numpy as np
import pandas as pd
import pandas_ta as ta
from google.cloud import bigquery
from moonshot.strategies.base import Moonshot
from numpy import nan
from pandas.testing import assert_frame_equal, assert_series_equal

if exists('/src'):
    WORKING_DIR = '/src'
else:
    with suppress(CalledProcessError):
        WORKING_DIR = subprocess.check_output(
            ['git', 'rev-parse', '--show-toplevel']).strip().decode()
        sys.path.extend([WORKING_DIR,
                         join(WORKING_DIR, 'src'),
                         join(WORKING_DIR, 'notebook'),
                         join(WORKING_DIR, 'notebook/*')])

        WORKING_DIR = join(WORKING_DIR, 'src')


warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.8f}'.format

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(StreamHandler())

In [2]:
from src.data_providers.data_provider import DataProvider
from src.data_providers.tradingview_provider import TradingViewProvider

GCLOUD_PROJECT = 'trading-strategy-develop'
REGION = 'asia-southeast1'

%set_env GCLOUD_PROJECT = $GCLOUD_PROJECT
%set_env REGION = $REGION

env: GCLOUD_PROJECT=trading-strategy-develop
env: REGION=asia-southeast1


In [3]:
provider: DataProvider = TradingViewProvider()
provider.WORKERS_NO = 4

In [4]:
# symbols = [
#     'SP:S5COND',
# ]

# fields = [
#     'description'
#     'price',
#     'change_5d',
#     'change_5d_pct',
#     'low_5d',
#     'high_5d',
# ]

# results = provider.quotes(symbols=symbols, fields=fields)

In [5]:
symbols = [
    'SP:S5COND',
]

ohlcv = provider.ohlcv(symbols=symbols,
                       freq='1D',
                       total_candles=252*5).reset_index()

In [6]:
provider.calc_perf(ohlcv, '5D')

{'SP:S5COND': {'close': 1463.72,
  'change_5d': 14.170000000000073,
  'change_5d_pct': 0.009775447552688816,
  'low_5d': 1439.81,
  'high_5d': 1473.95}}

In [8]:
provider.calc_perf(ohlcv, '1M')

{'SP:S5COND': {'close': 1463.72,
  'change_1m': 15.860000000000127,
  'change_1m_pct': 0.01095409777188411,
  'low_1m': 1402.29,
  'high_1m': 1473.95}}

In [9]:
provider.calc_perf(ohlcv, 'MTD')

{'SP:S5COND': {'close': 1463.72,
  'change_mtd': 40.22000000000003,
  'change_mtd_pct': 0.028254302774850738,
  'low_mtd': 1411.67,
  'high_mtd': 1473.95}}

In [10]:
provider.calc_perf(ohlcv, 'YTD')

{'SP:S5COND': {'close': 1463.72,
  'change_ytd': 45.6400000000001,
  'change_ytd_pct': 0.03218436195419166,
  'low_ytd': 1360.78,
  'high_ytd': 1493.28}}